In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb  8 15:45:44 2022

@author: ChewingGumKing_OJF
"""

#loads necessary libraries
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import os
from datetime import datetime
from datetime import date
import openpyxl
#import pyautogui
from selenium.webdriver.common.action_chains import ActionChains
#import shutil
#import glob
import os
import warnings
#import itertools
import json
import pandas as pd
import pyperclip as pc
from bs4 import BeautifulSoup as bs
import re
from random import randint
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
#import sys
warnings.filterwarnings("ignore")


In [2]:
#open a new window, load the website and logs in

driver = webdriver.Chrome(r'C:\Users\840 g3\Desktop\chromedriver.exe')


RL = 'https://www.akademie-heidelberg.de/online-seminare'
driver.get(RL)
try:
    driver.maximize_window()
except:
    pass

In [3]:
html = driver.page_source
soup = bs(html,'lxml')
allItems= soup.find_all('div',{'class':re.compile('views-row views-row-.+')})#.find_all('a')['href']
akademie=[]


In [4]:
for q in allItems:
    web=q.find('a')['href']
    link='https://www.akademie-heidelberg.de'+web
    ###################
    title=q.find('h2',{'class':'seminare__list__title'}).text.replace('\n','').strip()
    #####################################
#     date_time=q.find('ul',{'class':'seminare__list__list-date'}).text.replace('\n','').strip()
#     try:
#         on=q.find('span',{'class':'seminare__list__city'}).text.replace('\n','').strip()
#     except:
#         on=''
    ######################################    
    driver.get(link)
    try:
        htmln = driver.page_source
        soupn = bs(htmln,'lxml')
        sp= soupn.find('section',{'class':'representative'})
        cs=soupn.find('section',{'class':'seminare'})
        pr=soupn.find('section',{'class':'meta-data row border-top'})

    except WebDriverException:
        time.sleep(3)
        htmln = driver.page_source
        soupn = bs(htmln,'lxml')
        sp= soupn.find('section',{'class':'representative'})
        cs=soupn.find('section',{'class':'seminare'})
        pr=soupn.find('section',{'class':'meta-data row border-top'})
    #####################################
    ####################################
    try:
        trut=sp.find_all('div',{'class':'representative__block'})
        skerlist=[]
        for ma in trut:
            nam=ma.find('strong').text
            ti=ma.find('figcaption',{'class':'representative__name'})#.split('<br/>')[1]
            titl=str(ti).split('<br/>')[1]
            speakers={'name': nam,
                     'title':titl,
                     'link': ''}
            skerlist.append(speakers)
    except:
        skerlist=''
    if skerlist=='':
        Speakerlist=''
    else:
        Speakerlist=json.dumps(skerlist)
    ###################################################
    try:
        date_time=cs.find('span',{'class':'seminare__list__date'}).text.replace('\n','').strip()
    except AttributeError:
        date_time=''
    try:
        if date_time=='':
            start_date=end_date=time_st=''
        else:
            if '&'in date_time:
                start=date_time.split('&')[0]
                end=date_time.split('&')[1]

                start_date=start.split(',')[1].strip().replace('.','-')
                end_date=end.split(',')[1].strip().replace('.','-')

                sta_tim=start.split(',')[2].strip()#.replace(' - ',' ').split('-')[0]
                #en_tim=start.split(',')[2].strip().replace(' - ',' ').split('-')[1]
                if '-' in sta_tim:
                    shake=sta_tim.replace(' - ',' ').split(' ')
                    st_ti=shake[0]
                    spl_obj = datetime.strptime(st_ti, '%H:%M')
                    st_tim= spl_obj.strftime('%I:%M%p')
                    en_ti=shake[1]
                    spl_ti_obj = datetime.strptime(en_ti, '%H:%M')
                    en_tim= spl_ti_obj.strftime('%I:%M%p')
                    tim_z=shake[-1]
                else:
                    shake=sta_tim.split(' ')
                    st_ti=shake[0]
                    spl_obj = datetime.strptime(st_ti, '%H:%M')
                    st_tim= spl_obj.strftime('%I:%M%p')
                    en_tim=''
                    tim_z=shake[-1]

            elif len(date_time.split(','))<=3:
                date_ti=date_time.split(',')
                date_=date_ti[1].strip().replace('.','-')
                start_date=end_date=date_
                tim=date_ti[2].strip()
                if '-' in tim:
                    shake=tim.replace(' - ',' ').split(' ')
                    st_ti=shake[0]
                    spl_obj = datetime.strptime(st_ti, '%H:%M')
                    st_tim= spl_obj.strftime('%I:%M%p')
                    en_ti=shake[1]
                    spl_ti_obj = datetime.strptime(en_ti, '%H:%M')
                    en_tim= spl_ti_obj.strftime('%I:%M%p')
                    tim_z=shake[-1]
                else:
                    shake=tim.split(' ')
                    st_ti=shake[0]
                    spl_obj = datetime.strptime(st_ti, '%H:%M')
                    st_tim= spl_obj.strftime('%I:%M%p')
                    en_tim=''
                    tim_z=shake[-1]
            else:
                start=date_time.split('-')[0]
                end=date_time.split('-')[1]

                start_date=start.split(',')[1].strip().replace('.','-')
                end_date=end.split(',')[1].strip().replace('.','-')

                st_ti=start.split(',')[2].strip().split(' ')[0]
                
                spl_obj = datetime.strptime(st_ti, '%H:%M')
                st_tim= spl_obj.strftime('%I:%M%p')
                
                en_ti=end.split(',')[2].strip().split(' ')[0]
                
                spl_ti_obj = datetime.strptime(en_ti, '%H:%M')
                en_tim= spl_ti_obj.strftime('%I:%M%p')
                
                tim_z=start.split(',')[2].strip().split(' ')[1]
            time_st={'type': 'general',
                     'start_time':st_tim,
                     'end_time': en_tim,
                     'timezone': tim_z,
                     'days': 'all'}
    except:
        start_date=end_date=time_st=''
    if time_st=='':
        time_=''
    else:
        time_=json.dumps(time_st)
    #######################################
    event_info=''
    ###################################
    price=pr.find('div',{'class':'event__content'}).text.replace('\n','').strip()
    if ('kostenfreien' or 'Kostenfrei' or 'free') in price:
        ty='free'
        pri=''
        cur=''
    elif ('EUR' or 'eur' or 'Eur') in price:
        sel=price.split(',')[0].strip()
        selim=sel.split(' ')[1].strip()
        ty='paid'
        pri=selim
        cur='€'
    elif ('USD' or 'usd' or 'Usd') in price:
        sel=price.split(',')[0].strip()
        selim=sel.split(' ')[1].strip()
        ty='paid'
        pri=selim
        cur='$'

    else:
        ty=''
    if ty=='':
        ticket=''
    else:
        ticket={'type':ty,
                'price':pri,
                'currency':cur}
    if ticket=='':
        ticket_list=''
    else:
        ticket_list=json.dumps(ticket)
    ###################################
    
    org_pro="""Das Team der Akademie Heidelberg ist mit über 20-jähriger fundierter Erfahrung Ihr 
                Partner in Sachen gezielter Fort- und Weiterbildung. Speziell für Sie selektieren 
                wir in einem komplexen Umfeld die entscheidenden Informationen und setzen sie in 
                exakt auf Ihren Bedarf zugeschnittene Veranstaltungen um"""
    org_name='Akademie für Fortbildung Heidelberg'
    org_web='https://www.akademie-heidelberg.de/'
    logo=''
    sponsor=''
    agendalist=''
    type_=''
    category=''
    ###############################################
    
    def get_google_map_url(location):
        try:
            google_url_for_location="https://www.google.com/search?q="+location+"&oq="+location+"&num=1"
            time.sleep(randint(0,3))
            driver.get(google_url_for_location)
            google_map_url=driver.find_element_by_id('lu_map').click()
            time.sleep(1)
            google_map_url=driver.current_url
            return(google_map_url)
        except Exception as e:
            print(e)
            return("")
    #######################################
    try:
        on=cs.find('h2',{'class':'seminare__list__title'}).text.replace('\n','').strip()
    
        if ('Online' or 'Webinar') in on:
            ven=''
        else:
            try:
                ven=cs.find('span',{'class':'title-subsection seminare__subsection__title--light'}).text.replace('\n','').strip()
            except:
                ven=''
    except:
        ven=''
    ################################
    
    mail_=''
    ################################
    try:
        wi=cs.find('a')['href']
        event_web='https://www.akademie-heidelberg.de'+wi
    except:
        event_web=link
    #############################
    try:
        on=cs.find('h2',{'class':'seminare__list__title'}).text.replace('\n','').strip()
        if ('Online' or 'Webinar') in on:
            on_off='1'
        else:
            on_off='0'
    except:
        on_off='1'
    #####################################################
    if ven=='':
        city=''
        country=''
        venue=''
        googlePlaceUrl=''
    else:
        city=ven
        country='Gemany'
        venue=ven
        googlePlaceUrl=get_google_map_url(ven)
    
    akademie.append([link,title,start_date,end_date,time_,event_info,ticket_list,
                     org_pro,org_name,org_web,logo,sponsor,agendalist,
                     type_,category,city,country,venue,event_web,googlePlaceUrl,mail_,
                     Speakerlist,on_off])

In [5]:
akademie_dict=akademie

akademie_df= pd.DataFrame(columns=['scrappedUrl','eventname','startdate','enddate','timing','eventinfo','ticketlist','orgProfile','orgName','orgWeb','logo','sponsor','agendalist','type','category','city','country','venue','event_website','googlePlaceUrl','ContactMail','Speakerlist','online_event'],data=akademie_dict)
akademie_df.to_csv("akademie.tsv", sep = '\t',index=False)
